In [1]:
import numpy as np

In [2]:
doc_vec = np.load('doc_vec.npy')

In [3]:
N, D = doc_vec.shape
N, D

(28394, 300)

double for loop

too slow

In [ ]:
%%time
dists = []
for i in range(N):
    doc_dist = []
    for j in range(N):
        dist = np.linalg.norm(doc_vec[i, :] - doc_vec[j, :], ord=2)
        doc_dist.append(dist)
    dists.append(doc_dist)

dists = np.array(dists)

In [ ]:
dists.shape

single for loop

possible

In [ ]:
%%time
dists = []
for i in range(N):
    repeated_a = np.tile(doc_vec[i, :], (N, 1))
    dist = np.linalg.norm(repeated_a - doc_vec, ord=2, axis=1)
    dists.append(dist)

dists = np.vstack(dists)

no loop

oom. left requires 900GB!

```
MemoryError: Unable to allocate 901. GiB for an array with shape (28394, 8518200) and data type float32
```

The best is to use broadcast

In [4]:
left = np.tile(doc_vec.transpose(), (N,1)).reshape(N,D,N)
right = np.tile(doc_vec, (1,N)).reshape(N,D,N)

MemoryError: Unable to allocate 901. GiB for an array with shape (28394, 8518200) and data type float32

In [ ]:
dists = np.linalg.norm(left-right, ord=2, axis=1)

no loop broadcast right

In [ ]:
dists = np.linalg.norm(left-doc_vec[:, :, np.newaxis], ord=2, axis=1)

In [5]:
def mat_to_bytes(nrows, ncols, ndepth=None, dtype=32, out="GB"):
    """Calculate the size of a numpy array in bytes.
    :param nrows: the number of rows of the matrix.
    :param ncols: the number of columns of the matrix.
    :param dtype: the size of each element in the matrix. Defaults to 32bits.
    :param out: the output unit. Defaults to gigabytes (GB)
    :returns: the size of the matrix in the given unit
    :rtype: a float
    """
    sizes = {v: i for i, v in enumerate("BYTES KB MB GB TB".split())}
    if ndepth:
        return nrows * ncols * ndepth * dtype / 8 / 1024. ** sizes[out]
    else:
        return nrows * ncols * dtype / 8 / 1024. ** sizes[out]

In [6]:
mat_to_bytes(N, D, N)

901.0202094912529